In [ ]:


import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta
from typing import List, Union
from tqdm.notebook import tqdm



In [ ]:


def gerar_base_historica_ficticia():

    # Inicializa o gerador de dados falsos
    faker = Faker('pt_BR')
    
    # Define o número de linhas que você quer gerar
    n_pessoas = 1000
    
    # Gera os dados de pessoas com municípios
    dados_pessoas = {
        'uf': np.random.choice(['SP', 'RJ', 'MG', 'RS', 'BA'], size=n_pessoas),
        'municipio': [faker.city() for _ in range(n_pessoas)],
        'nome': [faker.name() for _ in range(n_pessoas)],
        'data_inicio': [faker.date_between(start_date='-5y', end_date='today') for _ in range(n_pessoas)],
        'data_fim': [faker.date_between(start_date='today', end_date='+5y') for _ in range(n_pessoas)],
    }
    
    # Cria o DataFrame de pessoas
    df_pessoas = pd.DataFrame(dados_pessoas)
    
    # Converte para datetime
    df_pessoas['data_inicio'] = pd.to_datetime(df_pessoas['data_inicio'])
    df_pessoas['data_fim'] = pd.to_datetime(df_pessoas['data_fim'])
    
    # Garante que data_inicio <= data_fim
    df_pessoas.loc[df_pessoas['data_inicio'] > df_pessoas['data_fim'], ['data_inicio', 'data_fim']] = \
        df_pessoas.loc[df_pessoas['data_inicio'] > df_pessoas['data_fim'], ['data_fim', 'data_inicio']].values

    return df_pessoas

    

In [ ]:


def gerar_tabela_sazonalidade(
    df: pd.DataFrame,
    sazonalidade: str,
    colunas_agrupamento: List[str],
    col_data_inicio: str,
    col_data_fim: str,
    col_contagem: str
) -> pd.DataFrame:

    display (f'Processando: {"-".join(colunas_agrupamento)} --- {sazonalidade}')
    
    colunas_necessarias = colunas_agrupamento + [col_data_inicio, col_data_fim]
    for col in colunas_necessarias:
        if col not in df.columns:
            raise ValueError(f"Coluna '{col}' não encontrada no DataFrame")
    
    df = df.copy()
    df[col_data_inicio] = pd.to_datetime(df[col_data_inicio])
    df[col_data_fim] = pd.to_datetime(df[col_data_fim])
    
    mask = df[col_data_inicio] > df[col_data_fim]
    df.loc[mask, [col_data_inicio, col_data_fim]] = df.loc[mask, [col_data_fim, col_data_inicio]].values
    
    resultados = []
    
    if sazonalidade == 'anual':
        ano_min = df[col_data_inicio].min().year
        ano_max = df[col_data_fim].max().year
        
        grupos = list(df[colunas_agrupamento].drop_duplicates().itertuples(index=False, name=None))
        for grupo in tqdm(grupos, desc='Processando grupos'):
            mask = True
            for col, val in zip(colunas_agrupamento, grupo):
                mask &= (df[col] == val)
            df_grupo = df[mask]
            
            for ano in range(ano_min, ano_max + 1):
                periodo_inicio = pd.Timestamp(year=ano, month=1, day=1)
                periodo_fim = pd.Timestamp(year=ano + 1, month=1, day=1)
                
                count = df_grupo[
                    (df_grupo[col_data_inicio] < periodo_fim) & 
                    (df_grupo[col_data_fim] >= periodo_inicio)
                ].shape[0]
                
                resultado = dict(zip(colunas_agrupamento, grupo))
                resultado.update({
                    'data_inicio': periodo_inicio,
                    'data_fim': periodo_fim - timedelta(days=1),
                    'sazonalidade': 'anual',
                    col_contagem: count
                })
                resultados.append(resultado)
    
    elif sazonalidade == 'mensal':
        data_min = df[col_data_inicio].min().replace(day=1)
        data_max = df[col_data_fim].max().replace(day=1) + pd.offsets.MonthBegin(1)
        periodos = pd.date_range(start=data_min, end=data_max, freq='MS')
        
        grupos = list(df[colunas_agrupamento].drop_duplicates().itertuples(index=False, name=None))
        for grupo in tqdm(grupos, desc='Processando grupos'):
            mask = True
            for col, val in zip(colunas_agrupamento, grupo):
                mask &= (df[col] == val)
            df_grupo = df[mask]
            
            for periodo in periodos:
                periodo_fim = periodo + pd.offsets.MonthBegin(1)
                count = df_grupo[
                    (df_grupo[col_data_inicio] < periodo_fim) & 
                    (df_grupo[col_data_fim] >= periodo)
                ].shape[0]
                
                resultado = dict(zip(colunas_agrupamento, grupo))
                resultado.update({
                    'data_inicio': periodo,
                    'data_fim': periodo_fim - timedelta(days=1),
                    'sazonalidade': 'mensal',
                    col_contagem: count
                })
                resultados.append(resultado)
    
    elif sazonalidade == 'diaria':
        data_min = df[col_data_inicio].min().date()
        data_max = df[col_data_fim].max().date()
        periodos = pd.date_range(start=data_min, end=data_max, freq='D')
        
        grupos = list(df[colunas_agrupamento].drop_duplicates().itertuples(index=False, name=None))
        for grupo in tqdm(grupos, desc='Processando grupos'):
            mask = True
            for col, val in zip(colunas_agrupamento, grupo):
                mask &= (df[col] == val)
            df_grupo = df[mask]
            
            for periodo in periodos:
                count = df_grupo[
                    (df_grupo[col_data_inicio] <= periodo) & 
                    (df_grupo[col_data_fim] >= periodo)
                ].shape[0]
                
                resultado = dict(zip(colunas_agrupamento, grupo))
                resultado.update({
                    'data_inicio': periodo,
                    'data_fim': periodo,
                    'sazonalidade': 'diaria',
                    col_contagem: count
                })
                resultados.append(resultado)
    
    else:
        raise ValueError("Sazonalidade deve ser 'anual', 'mensal' ou 'diaria'")
    
    df_resultado = pd.DataFrame(resultados)
    colunas_ordenacao = colunas_agrupamento + ['data_inicio']
    df_resultado = df_resultado.sort_values(colunas_ordenacao)
    
    return df_resultado.reset_index(drop=True)

    

In [ ]:


def identificar_periodos_estaveis(
    df_sazonal: pd.DataFrame,    
    colunas_agrupamento: List[str],
    col_data_inicio: str,
    col_data_fim: str,
    col_contagem: str
):
    
    with tqdm(total=len(df_sazonal), desc='Identificando períodos estáveis') as pbar:
        colunas_ordenacao = colunas_agrupamento + [col_data_inicio]
        df_sazonal = df_sazonal.sort_values(colunas_ordenacao)
        
        condicoes_mudanca = [(df_sazonal[col_contagem] != df_sazonal.groupby(colunas_agrupamento)[col_contagem].shift(1))]
        
        for col in colunas_agrupamento:
            condicoes_mudanca.append(df_sazonal[col] != df_sazonal[col].shift(1))
        
        df_sazonal['mudanca'] = np.any(condicoes_mudanca, axis=0)
        df_sazonal['grupo'] = df_sazonal.groupby(colunas_agrupamento)['mudanca'].cumsum()
        
        colunas_agregacao = colunas_agrupamento + ['grupo', col_contagem]
        periodos_estaveis = df_sazonal.groupby(colunas_agregacao).agg({
            col_data_inicio: 'min',
            col_data_fim: 'max',
            'sazonalidade': 'first'
        }).reset_index()
        
        pbar.update(len(df_sazonal))
    
    colunas_saida = colunas_agrupamento + [col_data_inicio, col_data_fim, 'sazonalidade', col_contagem]
    return periodos_estaveis[colunas_saida]

    

In [ ]:


def gerar_totais(
    df: pd.DataFrame,
    colunas_agrupamento: List[str],
    col_data_inicio: str,
    col_data_fim: str,
    col_contagem: str
) -> pd.DataFrame:

    

    for i in range(len(colunas_agrupamento)):

        colunas_agrupamento_atual = colunas_agrupamento[:i+1]
    
        for sazonalidade in ['mensal']:        
            
            # A tabela de sazonalidade possui um registro para cada item de sazonalidade
            # possui também as colunas selecionadas para agrupamento
            # Por exemplo: para cada conjunto municipio/uf, será gerado um registro para cada mes caso seja usada a sazonalidade mensal.
            df_sazonal = gerar_tabela_sazonalidade(
                df = df,    
                sazonalidade = sazonalidade,
                colunas_agrupamento = colunas_agrupamento_atual,
                col_data_inicio = col_data_inicio,
                col_data_fim = col_data_fim,
                col_contagem = col_contagem)                
            
            # Depois de criada a tabela com os dados no formato sazonal, podemos procurar por mudanças na variável de contagem.
            # Cada mudança encontrada marca o final de um périodo que possui data_inicio e data_fim.
            # Dessa forma a mesma informação pode ser representada de forma mais suscinta.
            df_periodos_estaveis = identificar_periodos_estaveis(
                df_sazonal = df_sazonal,
                colunas_agrupamento = colunas_agrupamento_atual,
                col_data_inicio = col_data_inicio,
                col_data_fim = col_data_fim,
                col_contagem = col_contagem)    
        
            
            df_periodos_estaveis.to_parquet(f'../../dados/{"-".join(colunas_agrupamento_atual)}_{sazonalidade}.parquet', index=False)        
    
    

In [ ]:


df_pessoas = gerar_base_historica_ficticia()

gerar_totais(
    df = df_pessoas,    
    colunas_agrupamento = ['uf', 'municipio'],
    col_data_inicio = 'data_inicio',
    col_data_fim = 'data_fim',
    col_contagem = 'quantidade_pessoas')

